# Caso de estudio: Clasificación con ajuste de hiperparámetros automatizado

En este ejemplo se muestra la clasificación de un conjunto de datos empleando Máquinas de Soporte Vectorial y una metaheurística para seleccionar automáticamente los hiperparámetros según sea el caso.

## Descripción del conjunto de datos

El conjunto de datos se conoce como [Breast cancer wisconsin (diagnostic)](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic))
y es un conjunto de datos que contiene información de todo tipo sobre muestras de tumores provenientes de pacientes con y sin cáncer de mama.
Este conjunto de datos es pequeño, con aproximadamente 600 datos, y es un problema de _clasificación binaria_, donde solamente se va a distinguir entre dos
posibles tipos de tumores: **benigno** o **maligno**.

La información del conjunto de datos se puede ver en el sitio oficial (usando el hipervínculo anterior), pero se da una descripción muy breve en este documento.
Se tienen 30 datos, entre los cuales son perímetro, área, concavidad, suavidad, entre otras cosas. En total son 10 características, pero se repiten por 3 variaciones
de células dentro del tumor, haciendo que el número total de características sea 30.

## Metodología y procedimiento

Para resolver este problema de clasificación se emplearán Máquinas de Soporte Vectorial empleando un kernel de tipo RBF. Con esto se tienen dos hiperparámetros a ajustar, $C$ y $\gamma$,
sin embargo, en este caso no se sabe nada acerca del conjunto de datos, por lo que no se puede emplear una técnica de búsqueda exhaustiva para encontrar los mejores valores.
Es por esto que se propone emplear una metaheurística que permita buscar rápidamente en el espacio de soluciones. Para esto se escogió el _Firefly algorithm_ como metaheurística.

El procedimiento general es el siguiente:

1. Procesar el conjunto de datos y estandarizarlo.
2. Separar el conjunto de datos en 85% entrenamiento y 15% prueba.
3. Iterar sobre el conjunto de entrenamiento utilizando validación cruzada estratificada de 10 pliegues para maximizar la precisión de clasificación, empleando Firefly algorithm para encontrar los mejores
    hiperparámetros.
4. Probar el resultado del clasificador con los datos de prueba y mostrar un reporte de clasificación.

In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Cargar el algoritmo completo
%run "../firefly.py"

En este paso se debe crear una función de tipo **caja negra** donde la función a optimizar solamente reciba un conjunto de valores (en este caso los hiperparámetros) y devuelva un valor (en este caso, la precisión de clasificación). La función de tipo **caja negra** lo que tiene dentro es una instancia de una Máquina de Soporte Vectorial que se ajusta con los datos que se le pasan a la función y devuelve el negativo (para minimizar) de la precisión encontrada.

In [3]:
def svc_fnc(x, x_tr=None, x_ts=None, y_tr=None, y_ts=None):
    # Crear una instancia del clasificador
    clf = SVC(C=x[0], gamma=x[1])
    # Ajustarlo con los datos de entrenamiento
    clf.fit(x_tr, y_tr)
    # Siempre se buscan valores positivos del accuracy
    score = -clf.score(x_ts, y_ts)

    return score

Algo importante que se debe realizar es _estandarizar_ los datos por dos razones fundamentales:

1. Las Máquinas de Soporte Vectorial solamente funcionan con datos estandarizados.

2. El conjunto de datos contiene diferentes escalas de longitud que van a afectar mucho al resultado final.

Es por esto que se decidió estandarizar los datos entre -1 y 1, de tal forma que las escalas de medición empleadas se ignoran, y la Máquina de Soporte Vectorial puede ser entrenada correctamente.

In [4]:
# Extraer el conjunto de datos, y separarlo en dos
X, y = load_breast_cancer(return_X_y=True)

# Estandarizar los datos entre -1 y 1
scaler = MinMaxScaler(feature_range=(-1, 1))
X = scaler.fit_transform(X)

In [5]:
# Es importante separar el conjunto de datos en prueba y entrenamiento
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

# Separar el conjunto de datos para validación cruzada usando
# separación estratificada de 10 pliegues
n_pliegues = 10
skf = StratifiedKFold(n_splits=n_pliegues)

Este es el ciclo principal del procedimiento, donde se separa el conjunto de entrenamiento en 10 partes, manteniendo un equilibrio entre ambas clases (la estratificación) y en cada paso iterativo se emplea el algortimo de optimización _Firefly_. Este algoritmo devuelve un conjunto de valores que serán los hiperparámetros que mejor minimizan la función de tipo **caja negra**, y al final se hará un promedio de los 10 pasos que se realizaron para tomar esta media como el resultado final.

In [15]:
%%time
# Estos arreglos guardarán los resultados finales
# Este arreglo guarda los parámetros óptimos, C y gamma
res_vals = np.zeros(2)
# Este arreglo guarda el valor de precisión total
fnc_total = np.array([])

# Se comienza a iterar sobre los 10 pliegues de la validación cruzada
for tr, ts in skf.split(x_train, y_train):
    # Estos son los parámetros de entrada del Firefly Algorithm
    kwargs = {'func': svc_fnc, 'dim': 2, 'tam_pob': 50, 'alpha': 0.9, 'beta': 0.2, 'gamma': 1.0, 
    'inf': 2**(-5), 'sup': 2**5}
    # Se crea una instancia del Firefly Algorithm
    fa_solve = FAOpt(**kwargs, args=(X[tr], X[ts], y[tr], y[ts]))
    # Se llama al método que resuelve la optimización (FA)
    fa_res, fa_fnc = fa_solve.optimizar(20, optim=True)

    # Se guardan los resultados de cada iteración
    res_vals += fa_res
    fnc_total = np.append(fnc_total, fa_fnc)

CPU times: user 29.9 s, sys: 42.9 ms, total: 29.9 s
Wall time: 29.9 s


Algo importante a notar es que en esta implementación del _Firefly algorithm_ se especifica un solo rango de búsqueda para todos los parámetros, y esto no siempre es deseable dado que diferentes parámetros tienen diferentes espacios de búsqueda. Adicionalmente, entre más grande el espacio de búsqueda parece ser que el _Firefly algorithm_ converge a valores muy grandes que reducen mucho la precisión. Si el lector lo desea, se puede tomar esta libreta y ejecutar, cambiando estos resultados.

In [13]:
# Los valores de los parámetros C y gamma deben estar normalizados, se divide por
# el número de pliegues
res_vals /= n_pliegues
# Por visualización, se muestran los resultados óptimos (opcional)
print(res_vals)
# Y el promedio de accuracy durante la validación cruzada (opcional)
print(fnc_total.mean())

# Por último, usando los valores óptimos encontrados se realiza la clasificación final
clf = SVC(C=res_vals[0], gamma=res_vals[1])
clf.fit(x_train, y_train)
print(clf.score(x_test, y_test))
# Y mostrar un reporte de clasificación
print(classification_report(y_test, clf.predict(x_test)))

[15.9253274   0.44476323]
0.9876700680272108
0.9651162790697675
              precision    recall  f1-score   support

           0       0.97      0.95      0.96        37
           1       0.96      0.98      0.97        49

   micro avg       0.97      0.97      0.97        86
   macro avg       0.97      0.96      0.96        86
weighted avg       0.97      0.97      0.97        86



Este resultado de clasificación es el esperado, aproximadamente se debe obtener un 98% de precisión con Máquinas de Soporte Vectorial; es posible que otros modelos/algoritmos tengan mejores resultados. En particular es interesante emplear las metaheurísticas para facilitar la búsqueda del espacio de soluciones y entrenar los modelos con este tipo de técnicas.